# Reinforcement learning for tic-tac-toe

In which we solve the problem of teaching a neural network to play tic-tac-toe.

* First, we import libraries for this example:

In [2]:
import random
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.backend import reshape
from keras.utils import to_categorical

* Next we write helper methods to initialize the board, get a list of valid moves, declare a winner, and print the current board.

In [3]:
# Get an empty board
#
# 0 indicates an empty space, 1 indicates an 'X' (player 1), and 2 indicates an 'O' (player 2)
#
# Initially the board is empty, so we return a 3x3 array of zeroes.
def initBoard():
    board = [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]
    return board

# Print the current state of the board
def printBoard(board):
    for i in range(len(board)):
        for j in range(len(board[i])):
            mark = ' '
            if board[i][j] == 1:
                mark = 'X'
            elif board[i][j] == 2:
                mark = 'O'
            if (j == len(board[i]) - 1):
                print(mark)
            else:
                print(str(mark) + "|", end='')
        if (i < len(board) - 1):
            print("-----")

# Get a list of valid moves (indices into the board)
def getMoves(board):
    moves = []
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 0:
                moves.append((i, j))
    return moves

# Declare a winner
#
# -1 = game not over
#  0 = draw
#  1 = 'X' wins (player 1)
#  2 = 'O' wins (player 2)
def getWinner(board):
    candidate = 0
    won = 0
    
    # Check rows
    for i in range(len(board)):
        candidate = 0
        for j in range(len(board[i])):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif j == len(board[i]) - 1:
                won = candidate
    
    if won > 0:
        return won
    
    # Check columns
    for j in range(len(board[0])):
        candidate = 0
        for i in range(len(board)):
            
            # Make sure there are no gaps
            if board[i][j] == 0:
                break
            
            # Identify the front-runner
            if candidate == 0:
                candidate = board[i][j]
            
            # Determine whether the front-runner has all the slots
            if candidate != board[i][j]:
                break
            elif i == len(board) - 1:
                won = candidate
    
    if won > 0:
        return won
    
    # Check diagonals
    candidate = 0
    for i in range(len(board)):
        if board[i][i] == 0:
            break
        if candidate == 0:
            candidate = board[i][i]
        if candidate != board[i][i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    candidate = 0
    for i in range(len(board)):
        if board[i][2 - i] == 0:
            break
        if candidate == 0:
            candidate = board[i][2 - i]
        if candidate != board[i][2 - i]:
            break
        elif i == len(board) - 1:
            won = candidate
    
    if won > 0:
        return won
    
    # Still no winner?
    if (len(getMoves(board)) == 0):
        # It's a draw
        return 0
    else:
        # Still more moves to make
        return -1

* Next, we test the helper methods to demonstrate that they work.

In [4]:
# Test helper methods
b = initBoard()
printBoard(b)
print(getWinner(b))
print(getMoves(b))

b[0][0] = 1
b[1][1] = 1
b[2][2] = 1
printBoard(b)
print(getWinner(b))

b[0][2] = 2
b[1][2] = 2
b[2][2] = 2
printBoard(b)
print(getWinner(b))

b[0][1] = 1
b[1][0] = 2
b[2][0] = 1
b[2][1] = 2
b[2][2] = 1
b[0][0] = 2
printBoard(b)
print(getWinner(b))

 | | 
-----
 | | 
-----
 | | 
-1
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
X| | 
-----
 |X| 
-----
 | |X
1
X| |O
-----
 |X|O
-----
 | |O
2
O|X|O
-----
O|X|O
-----
X|O|X
0


* Next, we take everything we've done so far and create a random game simulator

In [5]:
random.seed()

# Get best next move for the given player at the given board position
def bestMove(board, model, player, rnd=0):
    scores = []
    moves = getMoves(board)
    
    # Make predictions for each possible move
    for i in range(len(moves)):
        future = np.array(board)
        future[moves[i][0]][moves[i][1]] = player
        prediction = model.predict(future.reshape((-1, 9)), verbose=0)[0]
        if player == 1:
            winPrediction = prediction[1]
            lossPrediction = prediction[2]
        else:
            winPrediction = prediction[2]
            lossPrediction = prediction[1]
        drawPrediction = prediction[0]
        if winPrediction - lossPrediction > 0:
            scores.append(winPrediction - lossPrediction)
        else:
            scores.append(drawPrediction - lossPrediction)

    # Choose the best move with a random factor
    bestMoves = np.flip(np.argsort(scores))
    for i in range(len(bestMoves)):
        if random.random() * rnd < 0.5:
            return moves[bestMoves[i]]

    # Choose a move completely at random
    return moves[random.randint(0, len(moves) - 1)]

# Simulate a game
def simulateGame(p1=None, p2=None, rnd=0):
    history = []
    board = initBoard()
    playerToMove = 1
    
    while getWinner(board) == -1:
        
        # Chose a move (random or use a player model if provided)
        move = None
        if playerToMove == 1 and p1 != None:
            move = bestMove(board, p1, playerToMove, rnd)
        elif playerToMove == 2 and p2 != None:
            move = bestMove(board, p2, playerToMove, rnd)
        else:
            moves = getMoves(board)
            move = moves[random.randint(0, len(moves) - 1)]
        
        # Make the move
        board[move[0]][move[1]] = playerToMove
        
        # Add the move to the history
        history.append((playerToMove, move))
        
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
        
    return history

# Simulate a game
history = simulateGame()
print(history)

[(1, (0, 2)), (2, (0, 0)), (1, (2, 0)), (2, (1, 1)), (1, (2, 2)), (2, (0, 1)), (1, (2, 1))]


In [6]:
# Reconstruct the board from the move list
def movesToBoard(moves):
    board = initBoard()
    for move in moves:
        player = move[0]
        coords = move[1]
        board[coords[0]][coords[1]] = player
    return board

board = movesToBoard(history)
printBoard(board)
print(getWinner(board))

O|O|X
-----
 |O| 
-----
X|X|X
1


* Next, we generate a set of simulated games to train our neural network and calculate some win statistics for each random player. We predict that 'X' (player 1) should have a slight edge due to first mover advantage, but that both players have roughly the same number of wins, draws, and losses overall.

In [7]:
games = [simulateGame() for _ in range(10000)]

In [8]:
# Aggregate win/loss/draw stats for a player
def gameStats(games, player=1):
    stats = {"win": 0, "loss": 0, "draw": 0}
    for game in games:
        result = getWinner(movesToBoard(game))
        if result == -1:
            continue
        elif result == player:
            stats["win"] += 1
        elif result == 0:
            stats["draw"] += 1
        else:
            stats["loss"] += 1
    
    winPct = stats["win"] / len(games) * 100
    lossPct = stats["loss"] / len(games) * 100
    drawPct = stats["draw"] / len(games) * 100

    print("Results for player %d:" % (player))
    print("Wins: %d (%.1f%%)" % (stats["win"], winPct))
    print("Loss: %d (%.1f%%)" % (stats["loss"], lossPct))
    print("Draw: %d (%.1f%%)" % (stats["draw"], drawPct))

gameStats(games)
print()
gameStats(games, player=2)

Results for player 1:
Wins: 5837 (58.4%)
Loss: 2884 (28.8%)
Draw: 1279 (12.8%)

Results for player 2:
Wins: 2884 (28.8%)
Loss: 5837 (58.4%)
Draw: 1279 (12.8%)


* As shown above, when moves are chosen at random, player 1 has a significant advantage! However, we know that when people play tic-tac-toe, it's easy to force a draw, so we'd expect similar behavior from a neural network trained to play tic-tac-toe.
* We choose a DNN architecture since we effectively want to predict the outcome of a game based on the given board state.
* The input for each cell is the board state, which we reshape into a flat array of 9 elements, each element of which can be a 0 (empty cell), 1 (player 1 move), or 2 (player 2 move).
* The output is the result of the game (win, loss, or draw). We use a one-hot encoded array for this.

In [9]:
def getModel():
    numCells = 9 # How many cells in a 3x3 tic-tac-toe board?
    outcomes = 3 # How many outcomes are there in a game? (draw, X-wins, O-wins)
    model = Sequential()
    model.add(Dense(200, activation='relu', input_shape=(9, )))
    model.add(Dropout(0.2))
    model.add(Dense(125, activation='relu'))
    model.add(Dense(75, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(outcomes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model
model = getModel()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               2000      
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 125)               25125     
                                                                 
 dense_2 (Dense)             (None, 75)                9450      
                                                                 
 dropout_1 (Dropout)         (None, 75)                0         
                                                                 
 dense_3 (Dense)             (None, 25)                1900      
                                                                 
 dense_4 (Dense)             (None, 3)                 7

* Next, we reshape our data in preparation for training. 

In [10]:
# Get a set of board states labelled by who eventually won that game
def gamesToWinLossData(games):
    X = []
    y = []
    for game in games:
        winner = getWinner(movesToBoard(game))
        for move in range(len(game)):
            X.append(movesToBoard(game[:(move + 1)]))
            y.append(winner)

    X = np.array(X).reshape((-1, 9))
    y = to_categorical(y)
    
    # Return an appropriate train/test split
    trainNum = int(len(X) * 0.8)
    return (X[:trainNum], X[trainNum:], y[:trainNum], y[trainNum:])

# Split out train and validation data
X_train, X_test, y_train, y_test = gamesToWinLossData(games)

* Next, we train our model on the random tic-tac-toe games we generated earlier.

In [11]:
nEpochs = 100
batchSize = 100
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=nEpochs, batch_size=batchSize)

Epoch 1/100
610/610 [==============================] - 1s 1ms/step - loss: 0.9385 - acc: 0.5713 - val_loss: 0.8668 - val_acc: 0.6223
Epoch 2/100
610/610 [==============================] - 1s 910us/step - loss: 0.8812 - acc: 0.5967 - val_loss: 0.8546 - val_acc: 0.6126
Epoch 3/100
610/610 [==============================] - 1s 993us/step - loss: 0.8434 - acc: 0.6145 - val_loss: 0.7989 - val_acc: 0.6424
Epoch 4/100
610/610 [==============================] - 1s 966us/step - loss: 0.8169 - acc: 0.6237 - val_loss: 0.7866 - val_acc: 0.6507
Epoch 5/100
610/610 [==============================] - 1s 971us/step - loss: 0.7992 - acc: 0.6324 - val_loss: 0.7691 - val_acc: 0.6556
Epoch 6/100
610/610 [==============================] - 1s 969us/step - loss: 0.7868 - acc: 0.6379 - val_loss: 0.7459 - val_acc: 0.6670
Epoch 7/100
610/610 [==============================] - 1s 921us/step - loss: 0.7739 - acc: 0.6457 - val_loss: 0.7423 - val_acc: 0.6675
Epoch 8/100
610/610 [==============================] - 1s

* We see that our very simple model caps out at around 58% validation accuracy.
* Next, we use this trained model to play a game and see if we do any better than random chance.

* Now, we play a game against our model and see how well it does. We let it make the first move.

In [25]:
# Create new board
board = initBoard()

# Move 1 (computer)
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

 | | 
-----
 | | 
-----
 | |X


In [26]:
# Move 2 (human)
board[1][1] = 2
printBoard(board)

 | | 
-----
 |O| 
-----
 | |X


In [27]:
# Move 3 (computer)
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

X| | 
-----
 |O| 
-----
 | |X


In [28]:
# Move 4 (human)
board[0][2] = 2
printBoard(board)

X| |O
-----
 |O| 
-----
 | |X


In [29]:
# Move 5 (computer)
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

X| |O
-----
 |O| 
-----
X| |X


In [30]:
# Move 6 (human)
board[2][1] = 2
printBoard(board)

X| |O
-----
 |O| 
-----
X|O|X


In [31]:
# Move 7 (computer)
move = bestMove(board, model, 1)
board[move[0]][move[1]] = 1
printBoard(board)

X| |O
-----
X|O| 
-----
X|O|X


* Computer wins! This might be bad for humanity. :-)

# Convert to Cairo

### Export Keras Model Weights and Biases:

In [12]:
# Get weights and biases from the Keras model
weights_and_biases = {}

for layer in model.layers:
    weights = layer.get_weights()
    if len(weights) > 0:  # Ensure the layer has weights (Dense layers, Conv layers, etc.)
        layer_name = layer.name
        weights_and_biases[f"{layer_name}_weights"] = weights[0]
        if len(weights) > 1:  # Checking if biases exist
            weights_and_biases[f"{layer_name}_bias"] = weights[1]


In [13]:
import os
import numpy as np

# Create the directory if it doesn't exist
os.makedirs('src/generated', exist_ok=True)

for tensor_name, tensor in weights_and_biases.items():
    with open(os.path.join('src', 'generated', f"{tensor_name}.cairo"), "w") as f:
        f.write(
            "use array::ArrayTrait;\n\n" +
            "use orion::operators::tensor::{TensorTrait, Tensor, FP16x16Tensor};\n" +
            "use orion::numbers::{FixedTrait, FP16x16};\n\n" +

            "\nfn {0}() -> Tensor<FP16x16> ".format(tensor_name) + "{\n" +
            "    let mut shape = array!["
        )
        for dim in tensor.shape:
            f.write("{0},".format(dim))
        f.write(
            "].span();\n"
        )
        f.write(
            "    let mut data = array![\n"
        )
        for val in np.nditer(tensor.flatten()):
            f.write("FP16x16 {{ mag: {0}, sign: {1} }},".format(
                abs(int(val * 2**16)), str(val < 0).lower()))
        f.write(
            "].span();\n"
        )
        f.write(
            "    TensorTrait::new(shape, data)\n" +
            "}\n"
        )

with open(os.path.join('src', 'generated.cairo'), 'w') as f:
    for param_name in weights_and_biases.keys():
        f.write(f"mod {param_name};\n")
